## RAG Pipelines - Data Ingestion to Vector DB Pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

c:\Users\dmsuhaas\Downloads\RAG_with_LangChain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#read all pdfs inside directory
def process_all_pdfs(pdf_directory):
    all_documents=[]
    pdf_dir=Path(pdf_directory)
    pdf_files=list(pdf_dir.glob("**/*.pdf"))
    print(f"found {len(pdf_files)} pdf files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader=PyMuPDFLoader(str(pdf_file))
            documents=loader.load()

            #adding more info to the metadata
            for doc in documents:
                doc.metadata['source_file']=pdf_file.name
                doc.metadata['file_type']='pdf'

            all_documents.extend(documents)
            print(f"loaded {len(documents)} pages")

        except Exception as e:
            print(f"error: {e}")

    print(f"\nTotal document pages loaded: {len(all_documents)}")
    return all_documents

all_pdf_documents=process_all_pdfs("../data/pdf")

found 3 pdf files to process

Processing: code generation using LLMs (compressed).pdf
loaded 70 pages

Processing: flashfill.pdf
loaded 30 pages

Processing: Systematic mapping study of template based code generation.pdf
loaded 20 pages

Total document pages loaded: 120


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'creator': 'LaTeX with acmart 2021/05/01 v1.78 Typesetting articles for the Association for Computing Machinery and hyperref 2023-04-22 v7.00x Hypertext links for LaTeX', 'creationdate': '2024-11-12T01:58:59+00:00', 'source': '..\\data\\pdf\\code generation using LLMs (compressed).pdf', 'file_path': '..\\data\\pdf\\code generation using LLMs (compressed).pdf', 'total_pages': 70, 'format': 'PDF 1.5', 'title': 'A Survey on Large Language Models for Code Generation', 'author': '', 'subject': '-  General and reference  ->  Surveys and overviews.-  Software and its engineering  ->  Software development techniques.-  Computing methodologies  ->  Artificial intelligence.', 'keywords': '', 'moddate': '2024-11-12T01:58:59+00:00', 'trapped': '', 'modDate': 'D:20241112015859Z', 'creationDate': 'D:20241112015859Z', 'page': 0, 'source_file': 'code generation using LLMs (compressed).pdf'

In [4]:
#split text into chunks
#sliding window chunking
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n"," ",""]
    )
    split_docs=text_splitter.split_documents(documents)
    print(f"split {len(documents)} document pages into {len(split_docs)} chunks")

    return split_docs

In [5]:
chunks=split_documents(all_pdf_documents)

split 120 document pages into 644 chunks


In [6]:
chunks

[Document(metadata={'producer': 'pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'creator': 'LaTeX with acmart 2021/05/01 v1.78 Typesetting articles for the Association for Computing Machinery and hyperref 2023-04-22 v7.00x Hypertext links for LaTeX', 'creationdate': '2024-11-12T01:58:59+00:00', 'source': '..\\data\\pdf\\code generation using LLMs (compressed).pdf', 'file_path': '..\\data\\pdf\\code generation using LLMs (compressed).pdf', 'total_pages': 70, 'format': 'PDF 1.5', 'title': 'A Survey on Large Language Models for Code Generation', 'author': '', 'subject': '-  General and reference  ->  Surveys and overviews.-  Software and its engineering  ->  Software development techniques.-  Computing methodologies  ->  Artificial intelligence.', 'keywords': '', 'moddate': '2024-11-12T01:58:59+00:00', 'trapped': '', 'modDate': 'D:20241112015859Z', 'creationDate': 'D:20241112015859Z', 'page': 0, 'source_file': 'code generation using LLMs (compressed).pdf'

## Embeddings

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
class EmbeddingManager:
    def __init__(self, model_name:str="all-MiniLM-L6-v2"):
        self.model_name=model_name
        self.model=None
        self._load_model()

    def _load_model(self):
        try:
            print(f"loading embedding model: {self.model_name}")
            self.model=SentenceTransformer(self.model_name)
            print(f"model loaded successfully. embedding dimension: {self.model.get_sentence_embedding_dimension()}")

        except Exception as e:
            print(f"Error model model {self.model_name}:{e}")
            raise

    def generate_embeddings(self, texts: List[str]) ->np.ndarray:
        if not self.model:
            raise ValueError("model not loaded")
        
        print(f"generating embeddings for {len(texts)} texts")
        embeddings=self.model.encode(texts, show_progress_bar=True)
        print(f"generated embeddings with shape : {embeddings.shape}")
        return embeddings
    
embedding_manager=EmbeddingManager()
embedding_manager

loading embedding model: all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1003.70it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


model loaded successfully. embedding dimension: 384


## Vector Store

In [9]:
class VectorStore:
    def __init__(self, collection_name: str="pdf_documents", persist_directory: str = "../data/vector_store"):
        self.collection_name=collection_name
        self.persist_directory=persist_directory
        self.client=None
        self.collection=None
        self._initialize_store()

    def _initialize_store(self):
        try:
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            self.collection=self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={'description':"pdf document embeddings for RAG"}
        )
            print(f"vector store initialized. collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        if len(documents)!=len(embeddings):
            raise ValueError("number of documents must match number of embeddings")

        print(f"adding {len(documents)} documents to vector store")

        ids=[]
        metadatas=[]
        document_texts=[]
        embeddings_list=[]

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            doc_id=f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            metadata=dict(doc.metadata)
            metadata['doc_index']=i
            metadata['content_length']=len(doc.page_content)
            metadatas.append(metadata)
            document_texts.append(doc.page_content)
            embeddings_list.append(embedding.tolist())

        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=document_texts
            )
            print(f"successfully added {len(documents)} documents to vector store")
            print(f"total documents in collection : {self.collection.count()}")
        
        except Exception as e:
            print(f"error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

vector store initialized. collection: pdf_documents
Existing documents in collection: 0


In [10]:
chunks

[Document(metadata={'producer': 'pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'creator': 'LaTeX with acmart 2021/05/01 v1.78 Typesetting articles for the Association for Computing Machinery and hyperref 2023-04-22 v7.00x Hypertext links for LaTeX', 'creationdate': '2024-11-12T01:58:59+00:00', 'source': '..\\data\\pdf\\code generation using LLMs (compressed).pdf', 'file_path': '..\\data\\pdf\\code generation using LLMs (compressed).pdf', 'total_pages': 70, 'format': 'PDF 1.5', 'title': 'A Survey on Large Language Models for Code Generation', 'author': '', 'subject': '-  General and reference  ->  Surveys and overviews.-  Software and its engineering  ->  Software development techniques.-  Computing methodologies  ->  Artificial intelligence.', 'keywords': '', 'moddate': '2024-11-12T01:58:59+00:00', 'trapped': '', 'modDate': 'D:20241112015859Z', 'creationDate': 'D:20241112015859Z', 'page': 0, 'source_file': 'code generation using LLMs (compressed).pdf'

## convert text to embeddings

In [11]:
texts=[doc.page_content for doc in chunks]
texts

['1\nA Survey on Large Language Models for Code Generation\nJUYONG JIANG∗, The Hong Kong University of Science and Technology (Guangzhou), China\nFAN WANG∗, The Hong Kong University of Science and Technology (Guangzhou), China\nJIASI SHEN†, The Hong Kong University of Science and Technology, China\nSUNGJU KIM†, NAVER Cloud, South Korea\nSUNGHUN KIM†, The Hong Kong University of Science and Technology (Guangzhou), China\nLarge Language Models (LLMs) have garnered remarkable advancements across diverse code-related tasks,\nknown as Code LLMs, particularly in code generation that generates source code with LLM from natural\nlanguage descriptions. This burgeoning field has captured significant interest from both academic researchers\nand industry professionals due to its practical significance in software development, e.g., GitHub Copilot. Despite\nthe active exploration of LLMs for a variety of code tasks, either from the perspective of natural language',
 'the active exploration of LLMs 

In [12]:
#generate embeddings
embeddings =embedding_manager.generate_embeddings(texts)

#store the embeddings in vector database
vectorstore.add_documents(chunks, embeddings)

generating embeddings for 644 texts


Batches: 100%|██████████| 21/21 [00:08<00:00,  2.58it/s]


generated embeddings with shape : (644, 384)
adding 644 documents to vector store
successfully added 644 documents to vector store
total documents in collection : 644


## Retreiever pipeline from VectorStore

In [13]:
class RAGRetriever:
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query:str, top_k: int =5, score_threshold: float=0.0) -> List[Dict[str, Any]]:
        print(f"retrieving documents for query:'{query}'")
        print(f"Top K: {top_k}, Score Threshold: {score_threshold}")
        query_embedding =self.embedding_manager.generate_embeddings([query])[0]

        try:
            results=self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            retrieved_docs=[]

            if results['documents'] and results['documents'][0]:
                documents=results['documents'][0]
                metadatas=results['metadatas'][0]
                distances=results['distances'][0]
                ids=results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    similarity_score=1-distance

                    if similarity_score>=score_threshold:
                        retrieved_docs.append({
                        'id':doc_id,
                        'content':document,
                        'metadata':metadata,
                        'similarity_score':similarity_score,
                        'distance':distance,
                        'rank':i+1
                    })

                print(f"retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("no documents found")

            return retrieved_docs
    
        except Exception as e:
            print(f"error during retrieval: {e}")
            return []
    
rag_retriever=RAGRetriever(vectorstore, embedding_manager)

In [14]:
rag_retriever.retrieve("What are the characteristics of template-based code generation?")

retrieving documents for query:'What are the characteristics of template-based code generation?'
Top K: 5, Score Threshold: 0.0
generating embeddings for 1 texts


Batches: 100%|██████████| 1/1 [00:00<00:00, 120.23it/s]

generated embeddings with shape : (1, 384)
retrieved 5 documents (after filtering)


[{'id': 'doc_48aeebcf_552',
  'content': '2. What are the characteristics of template-based code generation approaches? We want to identify major characteristics of \nthese techniques and their tendencies. \n3. To what extent are template-based code generation tools being used in research? We are interested in identifying popular \ntools and their uses. \n4. What is the place of MDE in template-based code generation? We seek to determine whether and how MDE has inﬂuenced \nTBCG. \n3.2. Selection of source \nWe delimited the scope of the search to be regular publications that mention TBCG as at least one of the approaches \nused for code generation and published between 20 0 0–2016. Therefore, this includes publications where code generation is \nnot necessarily the main contribution. For example, Buchmann et al. [29] used TBCG to obtain ATL code while their main \nfocus was implementing a higher-order transformation. Given that not all publications have the term “code generation” in',


## Integrating VectorDB context pipeline with LLM output

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv(override=True)

# llm=ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     google_api_key=os.getenv("GOOGLE_API_KEY"),
#     temperature=0.1,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2
# )

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
#rag function - retrieve context + generate response
def rag(query, retriever, llm, top_k=6):
    #retrieve the context
    results=retriever.retrieve(query, top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question"
    
    #generate a response using gemini/groq
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context, query=query)])
    return response.content

In [17]:
answer=rag("what is flashfill", rag_retriever, llm)
answer

retrieving documents for query:'what is flashfill'
Top K: 6, Score Threshold: 0.0
generating embeddings for 1 texts


Batches: 100%|██████████| 1/1 [00:00<00:00, 107.47it/s]

generated embeddings with shape : (1, 384)
retrieved 5 documents (after filtering)


'FlashFill is a Programming by Example (PBE) system that can perform text manipulation operations, but has limitations in handling datetime and number operations, treating them as standard strings.'

## Enchanced RAG Pipeline features

In [18]:
#return answer, sources, confidence score and full context
def rag_advanced(query, retriever, llm, top_k=8, min_score=0.2, return_context=False):
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

result = rag_advanced("What are the characteristics of template-based code generation? Explain in detail", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

retrieving documents for query:'What are the characteristics of template-based code generation? Explain in detail'
Top K: 3, Score Threshold: 0.1
generating embeddings for 1 texts


Batches: 100%|██████████| 1/1 [00:00<00:00, 85.72it/s]

generated embeddings with shape : (1, 384)
retrieved 3 documents (after filtering)


Answer: The characteristics of template-based code generation (TBCG) can be identified as follows:

1. **Components**: TBCG consists of four main components:
   - **Data**: The input data used to generate the code.
   - **Template**: A predefined structure that contains placeholders for the data.
   - **Output**: The generated code, which is the result of combining the data and template.
   - **Meta-information**: The generation logic of the template, which is not explicitly mentioned by Jörges but is considered a crucial component.

2. **Template**: The template is a key component of TBCG, which favors reuse following the principle of "write once, produce many". This means that the same template can be used to generate multiple artifacts by varying the input data.

3. **Synthesis technique**: TBCG is considered a synthesis technique that uses templates to generate code. This approach requires less effort from programmers to develop code generators, making it an efficient method for co

## Advanced RAG Pipeline 

In [19]:
# Streaming, Citations, History, Summarization
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = [] 

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("What are the characteristics of template-based code generation? Explain in detail", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

retrieving documents for query:'What are the characteristics of template-based code generation? Explain in detail'
Top K: 3, Score Threshold: 0.1
generating embeddings for 1 texts


Batches: 100%|██████████| 1/1 [00:00<00:00, 132.26it/s]

generated embeddings with shape : (1, 384)
retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
2. What are the characteristics of template-based code generation approaches? We want to identify major characteristics of 
these techniques and their tendencies. 
3. To what extent are template-based code generation tools being used in research? We a

re interested in identifying popular 
tools and their uses. 
4. What is the place of MDE in template-based code generation? We seek to determine whether and how MDE has inﬂuenced 
TBCG. 
3.2. Selection of source 
We delimited the scope of the search to be regular publications that mention TBCG as at least one of the approaches 
used for code generation and published between 20 0 0–2016. Therefore, this includes publications where code generation is 
not necessarily the main contribution. For example, Buchmann et al. [29] used TBCG to obtain ATL code while their main 
focus was implementing a higher-order transformation. Given that not all publications have the term “code generation” in

Automatically generating code is an approach where the same generator can be reused to produce many different artifacts 
according to the varying inputs it receives. It also helps detecting errors in the input artifact early on before the generated 
code is compiled, when the output is source code. 
The